In [1]:
import json
import pandas as pd

def make_dataset(dir):
    # f1 = "working_60k/reranked.json"
    f1 = "working_6k_sci/reranked.json"
    f2 = "working_6k_nonsci/reranked.json"
    f3 = "working/reranked.json"

    columns = ['prompt', 'A', 'C', 'B', 'D', 'E', 'answer', 'tier_2_passages']
    
    train = [f1, f2]
    validation = [f3]
    train_dfs = [pd.DataFrame.from_records(json.load(open(f))) for f in train]
    train_df = pd.concat(train_dfs)
    train_df = train_df[columns]
    train_df.to_csv(f"{dir}/train.csv", index=False)

    val_dfs = [pd.DataFrame.from_records(json.load(open(f))) for f in validation]
    val_df = pd.concat(val_dfs)
    val_df = val_df[columns]
    # val_df['wikipedia_excerpt'] = None
    val_df.to_csv(f"{dir}/validation.csv", index=False)

make_dataset("kaggle_sci_qa/")

In [2]:
from datasets import load_dataset
swag = load_dataset("kaggle_sci_qa")

/home/apurva/miniconda3/envs/tf2/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
model_path = "sileod/deberta-v3-large-tasksource-nli"
# model_path = "deberta_ft/checkpoint-3005/"
# model_path = "microsoft/phi-1_5"

In [5]:
swag["train"][10]

{'prompt': 'What is the purpose of the citrate test?',
 'A': 'The citrate test is used to determine the ability of an organism to use oxaloacetate and acetate as a sole source of nitrogen.',
 'C': 'The citrate test is used to examine the ability of an organism to utilize citrate as the sole source of carbon and energy.',
 'B': 'The citrate test is used to detect the ability of an organism to utilize inorganic ammonium salts as a source of carbon and energy.',
 'D': 'The citrate test is used to identify the presence of the enzyme citrase in bacteria.',
 'E': 'The citrate test is used to assess the pH indicator bromothymol blue in a medium.',
 'answer': 'C',
 'tier_2_passages': '[{\'passage\': "It is useful for selecting for organisms that use citrate as its main carbon and energy source. It is a defined, selective and differential medium that tests for an organism\'s ability to use citrate as a sole carbon source and ammonium ions as the sole nitrogen source. After citrate enters a cell

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
from transformers import TrainingArguments, Trainer, AutoConfig, AutoModelForMultipleChoice

2023-09-22 01:20:46.948356: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-22 01:20:46.969065: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 01:20:47.313595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2023-09-22 01:20:48,235] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [11]:
# from modeling_mixformer_sequential import MixFormerSequentialForMultipleChoice
# config = AutoConfig.from_pretrained(model_path)
# config.pooler_hidden_size = 2048
# config.ignore_mismatched_sizes=True
# model = MixFormerSequentialForMultipleChoice.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForMultipleChoice.from_pretrained(model_path, ignore_mismatched_sizes=True)

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at sileod/deberta-v3-large-tasksource-nli and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([1, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import random
ABCDE = "ABCDE"
ending_names = [str(i) for i in range(5)]


def preprocess_function(examples):
    first_sentences = [[context] * 5 for context in examples["context"]]
    
    question_headers = examples["prompt"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # print(first_sentences, second_sentences)
    
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 5] for i in range(0, len(v), 5)] for k, v in tokenized_examples.items()}

# def keep_only_context(example):
#     return len(example['support']) > 0

def make_labels(example):
    answer = example['answer']
    example['label'] = ABCDE.index(answer)
    for i in range(5):
        example[str(i)] = example[ABCDE[i]]
    return example

def is_bad_passage(passage):
    words = passage.split(" ")
    lines = passage.split("\n")
    n_words_per_line = len(words) / len(lines)
    isbad = (len(lines) >= 5) and (n_words_per_line <= 5)
    if passage.count("|") > 10:
        isbad = True
    return isbad

def make_context(example):
    contexts = example["tier_2_passages"]
    contexts = eval(contexts)
    
    # extra_context = example.get('wikipedia_excerpt', '')
    # if extra_context:
    #     extra_title = extra_context.split(":", maxsplit=1)[0]
    #     extra_passage = extra_context.split(":", maxsplit=1)[1]
    
    #     extra_context = {'title':extra_title, 'passage':extra_passage}
    #     # print(extra_context)
    #     contexts = [extra_context] + contexts
    #     # contexts = [extra_context]
    
    openbook = ""
    max_openbook_len = 1024
    for context in contexts:
        tokens = tokenizer.encode(openbook)
        if len(tokens) > max_openbook_len:
            break
        passage = context['passage']
        if is_bad_passage(passage):
            continue
        title = context['title']
        passage = passage.replace("\n", " ")
        lpassage = len(tokenizer.encode(passage))
        if lpassage > 512:
            # print(passage)
            continue
        if lpassage + len(tokens) > max_openbook_len:
            continue
        openbook += f"""{title}: {passage}\n"""
        
    
    example['context'] = openbook
    return example

In [13]:
swag_context = swag.map(make_context)
swag_clean = swag_context.map(make_labels)
tokenized_swag = swag_clean.map(preprocess_function, batched=True)

Map:   0%|          | 0/12020 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/12020 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/12020 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [14]:
maxlen = 0
badguy = None
for elem in tokenized_swag['train']:
    for ids in elem['input_ids']:
        maxlen = max(len(ids), maxlen)
        if len(ids) > 3000:
            badguy = elem
print(maxlen)

1039


In [15]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [16]:
# import evaluate

# accuracy = evaluate.load("accuracy")

In [17]:
import numpy as np

def map_at_k(predictions, labels, k):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:k]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip(list(range(1,k+1)),x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

def compute_metrics(eval_pred):
    # predictions, labels = eval_pred
    # predictions = np.argmax(predictions, axis=1)
    # acc = accuracy.compute(predictions=predictions, references=labels)

    predictions = eval_pred.predictions.tolist()
    labels = eval_pred.label_ids.tolist()
    return {"map@3": map_at_k(predictions, labels, 3), "map@1": map_at_k(predictions, labels, 1)}

In [ ]:
requires_grad = False
layer_number = 0
for name, param in model.deberta.named_parameters():
    try:
        layer_number = name.split(".")[1]
        layer_number = int(layer_number)
        if layer_number > 26:
            requires_grad = True
    except:
        pass

    param.requires_grad = requires_grad

    print(name, layer_number, "trainable:", param.requires_grad, param.numel())

In [20]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for name, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || "
        f"all params: {all_param} || "
    )

print_trainable_parameters(model)

trainable params: 1050625 || all params: 435062785 || 


In [21]:
training_args = TrainingArguments(
    output_dir="deberta_ft2",
    save_strategy="epoch",
    # optim='adamw_bnb_8bit',
    # max_grad_norm=0.3,
    warmup_ratio=0.03,
    # load_best_model_at_end=True,
    # gradient_checkpointing=True,
    learning_rate=3e-4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,
    logging_steps=50,
    eval_steps=100,
    evaluation_strategy='steps',
    max_steps=61000,
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    push_to_hub=False,
    fp16=True,
    tf32=True,
    report_to="none"
)

tokenized_swag = tokenized_swag.shuffle()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_swag["train"],
    eval_dataset=tokenized_swag["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Map@3,Map@1
100,1.135700,0.559710,0.943333,0.910000
200,1.016200,0.328908,0.944167,0.910000
300,0.902700,0.300793,0.946667,0.915000
400,0.977800,0.342172,0.945833,0.910000
500,0.719800,0.283155,0.946667,0.915000
600,0.721700,0.285996,0.944167,0.910000
700,0.737600,0.290833,0.942500,0.905000
800,0.920900,0.290381,0.946667,0.915000
900,0.861700,0.278944,0.947500,0.915000
1000,0.731400,0.285776,0.944167,0.910000


In [ ]:
trainer.evaluate()

In [ ]:
tokens = tokenizer(["capital of paris is france. what is capital of france? paris",
                    "capital of paris is france. what is capital of france? delhi"], return_tensors='pt')

In [ ]:
test_predictions = trainer.predict(tokenized_swag["validation"]).predictions
predictions_as_ids = np.argsort(-test_predictions, 1)
predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]
predictions_as_string = test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]

In [ ]:
predictions_as_answer_letters

In [ ]:
val = pd.read_csv('kaggle_sci_qa/validation.csv')

In [ ]:
# val.to_dict(orient='records')

In [ ]:
print(swag_clean['validation'][0]['context'])

In [ ]:
print(model)